In [ ]:


import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os



In [ ]:
import matplotlib.pyplot as plt
import pydicom as dicom
import pydicom
from skimage.transform import resize
train='/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train'
arr=[]
for img in os.listdir(train):
    ds = pydicom.read_file(os.path.join(train,img))
    
    data=ds.pixel_array
    arr.append(data)
    resized_img = resize(data, (180, 180), anti_aliasing=True)
    #print(resized_img.shape)
    #print(label)
    plt.imshow(data)
    plt.show()
    #print(len(arr))
    break
  

In [ ]:

import pandas as pd
df=pd.read_csv('/kaggle/input/vinbigdata-chest-xray-abnormalities-detection/train.csv')
df.fillna(0,inplace=True)
df.head()

In [ ]:
df.info()

In [ ]:
df['class_id'].value_counts()

In [ ]:
from skimage import data, exposure, img_as_float
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib
from PIL import Image
import cv2
data=arr[0]
img_size=512

bbox = [df[df['image_id'] == '9a5094b2563a1ef3ff50dc5c7ff71345'].iloc[0,4],
        df[df['image_id'] == '9a5094b2563a1ef3ff50dc5c7ff71345'].iloc[0,5],
        df[df['image_id'] == '9a5094b2563a1ef3ff50dc5c7ff71345'].iloc[0,6],
        df[df['image_id'] == '9a5094b2563a1ef3ff50dc5c7ff71345'].iloc[0,7]]

image_array = cv2.resize(data , (img_size,img_size), interpolation = cv2.INTER_AREA)
image_array=exposure.equalize_adapthist(image_array/np.max(image_array))
fig, ax = plt.subplots(1, 1, figsize=(20, 4))


ax.imshow(image_array, cmap='gray')


c_height, c_width = data.shape[:2]
r_width = img_size / c_width
r_height = img_size / c_height


x = int(bbox[0] * r_width)
y = int(bbox[1] * r_height)
w = int((bbox[2]-bbox[0])* r_width)
h = int((bbox[3]-bbox[1]) * r_height)

p = matplotlib.patches.Rectangle((x, y),
                                 w,
                                 h,
                                 ec='g', fc='none', lw=2.)

data = exposure.equalize_hist(data/np.max(data))
ax.add_patch(p)
#plt.imshow(data,'gray')
plt.show()

In [ ]:
print(x,y,w,h)

In [ ]:
img = exposure.equalize_adapthist(data/np.max(data))
plt.figure(figsize = (7,7))
plt.imshow(img,'gray')
plt.show()

In [ ]:
from pydicom import dcmread
from pydicom.pixel_data_handlers.util import apply_voi_lut
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)
    
    # VOI LUT (if available by DICOM device) is used to transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    
        
    return data

In [ ]:
import time
import tqdm
import pydicom
import cv2
import albumentations
from skimage.transform import resize
import warnings
import os
import numpy as np
warnings.simplefilter("ignore", UserWarning)
# from pydicom.config import image_handlers
# print(image_handlers)
# image_handlers[2].have_pillow_jpeg2000_plugin = Fals

img_size=512
data_dir='../input/vinbigdata-chest-xray-abnormalities-detection/'

data_df=df
class_names = sorted(data_df['class_name'].unique())
print(f"No. of classes read - {len(class_names)}")
time.sleep(1)

images_list = sorted(os.listdir(os.path.join(data_dir, 'train')))
X = []
Y = []
b_box=[]


for image in tqdm.tqdm(images_list[:5000]):
    cls_index = data_df[data_df['image_id'] == image[:-6]].iloc[0,2]
    if cls_index !=14:
        x_min=df[df['image_id'] == image[:-6]].iloc[0,4]
        y_min=df[df['image_id'] == image[:-6]].iloc[0,5]
        x_max=df[df['image_id'] == image[:-6]].iloc[0,6]
        y_max=df[df['image_id'] == image[:-6]].iloc[0,7]
        image_path = os.path.join(data_dir, 'train',image)
        data=read_xray(image_path)
        image_array = cv2.resize(data , (img_size,img_size), interpolation = cv2.INTER_AREA)
#         image_array=exposure.equalize_adapthist(image_array/np.max(image_array))
        bboxes=[x_min,y_min,x_max,y_max]
        c_height, c_width = data.shape[:2]
        r_width = img_size / c_width
        r_height = img_size / c_height
        x = int(bboxes[0] * r_width)
        y = int(bboxes[1] * r_height)
        w = int((bboxes[2]-bboxes[0]) * r_width)
        h = int((bboxes[3]-bboxes[1]) * r_height)
        new_box =[x, y, w, h]
        b_box.append(new_box)
        X.append(image_array)
        Y.append(cls_index)
    else:
        continue

In [ ]:
for image in tqdm.tqdm(images_list[5000:10000]):
    cls_index = data_df[data_df['image_id'] == image[:-6]].iloc[0,2]
    if cls_index !=14:
        x_min=df[df['image_id'] == image[:-6]].iloc[0,4]
        y_min=df[df['image_id'] == image[:-6]].iloc[0,5]
        x_max=df[df['image_id'] == image[:-6]].iloc[0,6]
        y_max=df[df['image_id'] == image[:-6]].iloc[0,7]
        image_path = os.path.join(data_dir, 'train',image)
        data=read_xray(image_path)
        image_array = cv2.resize(data , (img_size,img_size), interpolation = cv2.INTER_AREA)
#         image_array=exposure.equalize_adapthist(image_array/np.max(image_array))
        bboxes=[x_min,y_min,x_max,y_max]
        c_height, c_width = data.shape[:2]
        r_width = img_size / c_width
        r_height = img_size / c_height
        x = int(bboxes[0] * r_width)
        y = int(bboxes[1] * r_height)
        w = int((bboxes[2]-bboxes[0]) * r_width)
        h = int((bboxes[3]-bboxes[1]) * r_height)
        new_box =[x, y, w, h]
        b_box.append(new_box)
        X.append(image_array)
        Y.append(cls_index)
    else:
        continue

In [ ]:
for image in tqdm.tqdm(images_list[10000:15000]):
    cls_index = data_df[data_df['image_id'] == image[:-6]].iloc[0,2]
    if cls_index !=14:
        x_min=df[df['image_id'] == image[:-6]].iloc[0,4]
        y_min=df[df['image_id'] == image[:-6]].iloc[0,5]
        x_max=df[df['image_id'] == image[:-6]].iloc[0,6]
        y_max=df[df['image_id'] == image[:-6]].iloc[0,7]
        image_path = os.path.join(data_dir, 'train',image)
        data=read_xray(image_path)
        image_array = cv2.resize(data , (img_size,img_size), interpolation = cv2.INTER_AREA)
#         image_array=exposure.equalize_adapthist(image_array/np.max(image_array))
        bboxes=[x_min,y_min,x_max,y_max]
        c_height, c_width = data.shape[:2]
        r_width = img_size / c_width
        r_height = img_size / c_height
        x = int(bboxes[0] * r_width)
        y = int(bboxes[1] * r_height)
        w = int((bboxes[2]-bboxes[0]) * r_width)
        h = int((bboxes[3]-bboxes[1]) * r_height)
        new_box =[x, y, w, h]
        b_box.append(new_box)
        X.append(image_array)
        Y.append(cls_index)
    else:
        continue

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(20, 4))


ax.imshow(image_array, cmap='gray')

p = matplotlib.patches.Rectangle((x, y),
                                 w,
                                 h,
                                 ec='g', fc='none', lw=2.)


ax.add_patch(p)
plt.show()


In [ ]:
from torchvision import transforms
import albumentations
transform=transforms.Compose([
        transforms.ToPILImage(),
        transforms.Grayscale(3),
        transforms.ToTensor(),
        ]) 

In [ ]:
from imgaug import augmenters as iaa
image_augmentations = iaa.Sequential( 

    [                                

    iaa.Affine(scale=(0.5, 1.5)),

    iaa.Affine(rotate=(-60, 60)),

    iaa.Affine(translate_percent={"x":(-0.3, 0.3),"y":(-0.3, 0.3)}),

    iaa.Fliplr(1),

    iaa.Multiply((0.5, 1.5)),

    iaa.GaussianBlur(sigma=(1.0, 3.0)),

    # Add Gaussian Noise

    iaa.AdditiveGaussianNoise(scale=(0.03*255, 0.05*255))

])


In [ ]:
Xarr = np.array(X,dtype=int)
Yarr = np.array(Y)

In [ ]:
np.unique(Y,  return_counts=True)

In [ ]:
print(len(X), len(Y))

In [ ]:
import gc
del(X)
print(Xarr.shape, Yarr.shape)
gc.collect()

In [ ]:
# from tensorflow.keras.utils import to_categorical
# Yarr_hot = to_categorical(Y)
# print(Xarr.shape, Yarr_hot.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model,Sequential
from tensorflow.keras.layers import Flatten,MaxPooling2D,Activation, Conv2D,BatchNormalization, Dense, GlobalAveragePooling2D, Lambda, Dropout, InputLayer, Input
from tensorflow.keras.utils import to_categorical


In [ ]:
print(tf.__version__)

In [ ]:
Xarr=Xarr/255.0

In [ ]:
Xarr = np.array(Xarr).reshape(-1, img_size, img_size,1)

In [ ]:
Xarr.shape[1:]

In [ ]:
model = tf.keras.applications.ResNet50( input_shape=(512,512,1), 
    include_top=False, 
    weights=None
    )

In [ ]:
# model=Sequential()
# model.add(Conv2D(256, (3, 3),  input_shape= Xarr.shape[1:]))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Conv2D(256, (3, 3)))
# model.add(Activation('relu'))
# model.add(MaxPooling2D(pool_size=(2, 2)))

# model.add(Flatten()) 
# model.add(Dense(240))

# model.add(Dense(Yarr.shape[-1]))
# model.add(Activation('softmax'))





In [ ]:
base_model_output=model.output

flattened_output = Flatten()(base_model_output)

class_prediction = Dense(256, activation="relu")(flattened_output)
class_prediction = Dense(256, activation="relu")(class_prediction )
class_prediction = Dropout(0.2)(class_prediction)
class_prediction = Dense(256, activation="relu")(class_prediction)
class_prediction = Dropout(0.2)(class_prediction )
class_prediction = Dense(128, activation="relu")(class_prediction)
class_prediction = Dense(15, activation='softmax',
                             name="class_output")(class_prediction)

box_output = Dense(256, activation="relu")(flattened_output)
box_output = Dense(256, activation="relu")(box_output)
box_output = Dropout(0.2)(box_output )
box_output = Dense(256, activation="relu")(box_output)
box_output = Dropout(0.2)(box_output )
box_output = Dense(32, activation="relu")(box_output)
box_predictions = Dense(4, activation='sigmoid',name= "box_output")(box_output)

In [ ]:
import tensorflow

In [ ]:
b_box=np.array(b_box).reshape( len(Y),-1)


In [ ]:
from sklearn.model_selection import train_test_split
train_images, val_images, train_labels, \
val_labels, train_boxes, val_boxes = train_test_split( Xarr, 
                Yarr, b_box, test_size = 0.1, 
                random_state = None)

print('Total Training Images: {}, Total Test Images: {}'.format(
    len(train_images), 
    len(val_images)
    ))

In [ ]:
op = Model(inputs=model.input, outputs= [box_predictions,class_prediction])


In [ ]:
op.summary()

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping
losses = { 
    "box_output": "MeanSquaredError",
    "class_output": "sparse_categorical_crossentropy"
    }

loss_weights = {
    "box_output": 2.0, 
    "class_output": 2.0
    }

metrics = {
    'class_output': 'accuracy', 
    'box_output':  'mse'
    }

stop = EarlyStopping(monitor = "val_loss", min_delta = 0.001, patience = 3, 
                    restore_best_weights = True
                     )


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.0002, 
                              patience = 30, min_lr = 1e-7, verbose = 1)


opt = tf.keras.optimizers.Adam()


op.compile(optimizer = opt, loss = losses, loss_weights = loss_weights, 
    metrics = metrics)

In [ ]:
from tensorflow.keras.applications import densenet
chex=densenet.DenseNet121(include_top=False,weights=None,input_shape=Xarr.shape[1:],pooling='avg')
# output=chex.output
base_model1_output=chex.output

flattened_output = Flatten()(base_model1_output)

class_prediction = Dense(256, activation="relu")(flattened_output)
class_prediction = Dense(256, activation="relu")(class_prediction )
class_prediction = Dropout(0.2)(class_prediction)
class_prediction = Dense(256, activation="relu")(class_prediction)
class_prediction = Dropout(0.2)(class_prediction )
class_prediction = Dense(128, activation="relu")(class_prediction)
class_prediction = Dense(14, activation='softmax',
                             name="class_output")(class_prediction)

box_output = Dense(256, activation="relu")(flattened_output)
box_output = Dense(256, activation="relu")(box_output)
box_output = Dropout(0.2)(box_output )
box_output = Dense(256, activation="relu")(box_output)
box_output = Dropout(0.2)(box_output )
box_output = Dense(32, activation="relu")(box_output)
box_predictions = Dense(4, activation='sigmoid',name= "box_output")(box_output)

In [ ]:
op1 = Model(inputs=chex.input, outputs= [box_predictions,class_prediction])

In [ ]:

from tensorflow.keras.callbacks import EarlyStopping
losses = { 
    "box_output": "MeanSquaredError",
    "class_output": "sparse_categorical_crossentropy"
    }

loss_weights = {
    "box_output": 1.0, 
    "class_output": 1.0
    }

metrics = {
    'class_output': 'accuracy', 
    'box_output':  'mse'
    }

stop = EarlyStopping(monitor = "val_loss", min_delta = 0.01, patience = 3, 
                    restore_best_weights = True
                     )


reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(monitor = "val_loss", factor = 0.0002, 
                              patience = 30, min_lr = 1e-7, verbose = 1)


opt = tf.keras.optimizers.Adam()


op1.compile(optimizer = opt, loss = losses, loss_weights = loss_weights, 
    metrics = metrics)

In [ ]:
# train_image22=ImageDataGenerator(image_augmentations)

In [ ]:
history1 = op1.fit(x=train_images,
                  y= ({
                        "box_output": train_boxes, 
                        "class_output": train_labels
                        }),
                      
                   
                    validation_data=(
                        val_images, 
                        {
                          "box_output": val_boxes, 
                          "class_output": val_labels
                          }), batch_size = 8, epochs = 10, 
                    callbacks=reduce_lr)

In [ ]:
print(history1.history.keys())
#  "Accuracy"
plt.plot(history1.history['class_output_accuracy'])
plt.plot(history1.history['val_class_output_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()
# "Loss"
plt.plot(history1.history['class_output_loss'])
plt.plot(history1.history['val_class_output_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [ ]:
data_dir='../input/vinbigdata-chest-xray-abnormalities-detection/'


class_names = sorted(data_df['class_name'].unique())
print(f"No. of classes read - {len(class_names)}")
time.sleep(1)

images_list = sorted(os.listdir(os.path.join(data_dir, 'test')))
test1 = []
cs=[]


for image in tqdm.tqdm(images_list):
    image_path = os.path.join(data_dir, 'test',image)
    data=read_xray(image_path)
#     real = exposure.equalize_adapthist(data/np.max(data))
    image_array = cv2.resize(data , (img_size,img_size), interpolation = cv2.INTER_AREA)
    test = np.array(image_array).reshape(-1, img_size, img_size,1)
    test=test/255.0
#     test1.append(test)
    result=op1.predict(test)
    result1=image[:-6],str(np.argmax(result[1]))+' '+str(np.max(result[1]))+' '+str(result[0][0][0])+' '+str(result[0][0][1])+' '+str(result[0][0][2])+' '+str(result[0][0][3])
    cs.append(result1)
#     print(result)
#     test.append(image_array)
    

In [ ]:
# cs=[]
# for i in test1:
    
#     result=op1.predict(i)
#     result1=image[:-6],str(np.argmax(result[1]))+' '+str(np.max(result[1]))+' '+str(result[0][0][0])+' '+str(result[0][0][1])+' '+str(result[0][0][2])+' '+str(result[0][0][3])
#     cs.append(result1)
    

In [ ]:
# cs=np.array(cs)


In [ ]:
cs

In [ ]:
from pandas import DataFrame
cs = DataFrame (cs,columns=['image_id','PredictionString'])


In [ ]:
cs

In [ ]:
cs.to_csv('./test1.csv',index=False)

In [ ]:
xxx=pd.read_csv("./test1.csv")
xxx

In [ ]:
submission=pd.read_csv('../input/vinbigdata-chest-xray-abnormalities-detection/sample_submission.csv')
submission